### **Preparing Input Text for LLM**
There are four steps involved in preparing input text for use by large language models: 
1. Splitting text into individual word and subword tokens (Tokenization)
2. Convert tokens into token IDs. 
3. Generate input-target pairs. 
4. Encoding the tokenized text into vector representations (Embedding) 

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/04.webp" width="600px">

### **1. Tokenizing Text**
There are several simple ways to tokenize text. 
1. Split a text on whitespace characters. 
2. Split a text on whitespace and punctuations. 
3. Split a text on whitespace and punctuations and remove redundant whitspaces. 

In [1]:
import re 
text = "Hello, world. This, is a test."

# 1. Split on whitespace characters 
result = re.split(r'(\s)', text)
print("Splitting on whitespace characters: ", result)

# 2. Split a text on whitespace and punctuations
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
print("Splitting on whitespace and punctuation: ", result)

# 3. Split a text on whitespace and punctuations and remove redundant whitspaces
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print("Splitting on whitespace and punctuation and removing redundant whitespace: ", result)

Splitting on whitespace characters:  ['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']
Splitting on whitespace and punctuation:  ['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']
Splitting on whitespace and punctuation and removing redundant whitespace:  ['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


We apply the above tokenization techniques to a short story by Edith Wharton called "The Verdict". 

In [3]:
# Loading short story 
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
print("Total number of character:", len(raw_text))

# Apply tokenization technique 
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30]) # Visualise first 30 tokens 

Total number of character: 20479
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


### **2. Converting Tokens into Token IDs**
- To map the tokens into token IDs, we first build a vocabulary. The vocabulary defines how we map each unique word and special character to a unique integer. 

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/06.webp" width="500px">

In [4]:
# Create a list of unique tokens and sort alphabetically to determine vocabulary size 
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print("Vocabulary size: ", vocab_size)

# Print first 50 entries in vocabulary 
vocab = {token:integer for integer,token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

Vocabulary size:  1130
('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [ ]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    # Splits text into tokens and carries out the string-to-integer mapping to produce token IDs 
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    # Carries out the reverse integer-to-string mapping to convert the token IDs back to text 
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/08.webp?123" width="500px">

In [7]:
tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print("Token IDs after encoding: ", ids)
print("Text after decoding token IDs: ", tokenizer.decode(ids))

Token IDs after encoding:  [1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]
Text after decoding token IDs:  " It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


- However, the above tokenizer cannot be generalized to text that contains tokens not in the vocabulary training set. Hence, we need to add special context tokens. 

In [8]:
tokenizer = SimpleTokenizerV1(vocab)
text = "Hello, do you like tea. Is this-- a test?"
tokenizer.encode(text)

KeyError: 'Hello'

Some of these special tokens are
  - `[BOS]` (beginning of sequence) marks the beginning of text.
  - `[EOS]` (end of sequence)  or `[endoftext]` marks where the text ends (this is usually used to concatenate multiple unrelated texts, e.g., two different Wikipedia articles or two different books, and so on).
  - `[PAD]` (padding) if we train LLMs with a batch size greater than 1 (we may include multiple texts with different lengths; with the padding token we pad the shorter texts to the longest length so that all texts have an equal length).
- `[UNK]` to represent words that are not included in the vocabulary.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/10.webp" width="500px">

In [10]:
# Add <|endoftext|> and <|unk|> to vocabulary 
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer,token in enumerate(all_tokens)}

# Modified tokenizer 
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int 
            else "<|unk|>" for item in preprocessed # Map token to <|unk|> if token is not vocabulary 
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

tokenizer = SimpleTokenizerV2(vocab)
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
print(text)
print("Encoded token IDs: ", tokenizer.encode(text)) 
print("Decoded tokens: ", tokenizer.decode(tokenizer.encode(text)))

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.
Encoded token IDs:  [1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]
Decoded tokens:  <|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


Note that "Hello" and "palace" were not in the vocabulary list and hence, they were mapped to `<|unk|>`. 

#### **Byte Pair Encoding**
- The byte pair encoding tokenizer was used to train LLMs such as GPT-2, GPT-3 and the original model used in ChatGPT. 
- The BPE tokenizer has a total vocabulary size of 50257. 
- The BPE tokenizer can encode and decode any unknown words without using `<|unk|>` tokens. 
- The algorithm breaks down words that are not in its predefined vocabulary into smaller subword units or individual characters, enabling BPE to handle out-of-vocabulary words. 

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/11.webp" width="500px">

In [11]:
import importlib
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

# Use byte-pair encoding to encode token IDs 
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace.")
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print("Token IDs using byte pair encoding: ", integers)

# Decode token IDs 
print("Decoded token IDs: ", tokenizer.decode(integers))

Token IDs using byte pair encoding:  [15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]
Decoded token IDs:  Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


### **3. Generate input-target pairs**
- LLMs are pretrained by predicting the next word in a text. 

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/12.webp" width="400px">

- One of the easiest ways to create the input-target pairs for the next-word prediction task is to create two variables `x` and `y`, where `x` contains the input tokens and `y` contains the targets, which are the inputs shifted by 1.

In [15]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
enc_text = tokenizer.encode(raw_text)
enc_sample = enc_text[50:]

# Input-target pair by shifting input by 1. 
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x: {x}")
print(f"y: {y}")

# Visualising tokens used to predict a the next token 
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))
    print("")

x: [290, 4920, 2241, 287]
y: [4920, 2241, 287, 257]
[290] ----> 4920
 and ---->  established

[290, 4920] ----> 2241
 and established ---->  himself

[290, 4920, 2241] ----> 287
 and established himself ---->  in

[290, 4920, 2241, 287] ----> 257
 and established himself in ---->  a



- An efficient data loader that iterates over the input dataet and returns the inputs and targets as PyTorch tensors is implemented. 

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/13.webp?123" width="500px">

In [19]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers)

    return dataloader

The `stride` setting dictates the number of positions the inputs shift across batches, emulating a sliding window approach. 
Strides can be increased to prevent overlaps between batches, since more overlap can lead to increased overfitting.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/14.webp" width="500px">

In [22]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print("First batch: ", first_batch)
second_batch = next(data_iter)
print("Second batch: ", second_batch)

First batch:  [tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
Second batch:  [tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


Note that an input size of 4 is small and it is common to train LLMs with input sizes of at least 256. 

In [23]:
# Increasing stride to 4

dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


### **4. Creating Token Embeddings**
- As a preliminary step, we initialize embedding weights with random values. The initialization serves as a starting point for the LLM's learning process. We will optimize the embedding weights as part of the LLM training. 

In [24]:
# Suppose a vocabulary size of 6 and an embedding of size 3 
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(6, 3)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


There is one row of embedding for each of the six tokens in the vocabulary. Lookup operation can be used to obtain the embedding for a particular token.

#### **Encoding Word Positions**
- A minor shortcoming of LLMs is that their self-attention mechanism does not have a notion of position or oder for the tokens within a sequence. 
- The previous embedding maps the same token ID to the same vector representation, regardless of its position in the input sequence. 

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/17.webp" width="500px">

- To inject position information into the LLM, we can use two broad categories of position-aware embeddings: relative positional embeddings and absolute positional embeddings. 
- For each position in the input sequence, a unique embedding is added to the token's embedding to convey its exact location. 

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/18.webp" width="500px">

- Absolute positional embeddings are associated with specific positions in a sequence. Relative positional embeddings focus on the relative position or distance between tokens. Relative positional embeddings learns the relationships in terms of "how far apart" instead of "at which position". 
- The advantage of relative positional embedding is that it can generalize better to sequences of varying lengths, even if it has not seen such lengths during training. 

In [ ]:
# Embedding for a vocabulary size of 50257 and size 256 
token_embedding_layer = torch.nn.Embedding(50257, 256)

max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=4, 
    stride=max_length, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs) 
print("\nInputs shape:\n", inputs.shape)

token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape) # Batch size of 8 with 4 tokens each and each token is embedded into a 256-dimensional vector. 

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])
torch.Size([8, 4, 256])


- For a GPT model's absolute embedding approach, we create another embedding layer that has the same embedding dimension as the token embedding layer. 
- The input to the `pos_embedding` is a placeholder vector `torch.arange(context_length)`, which contains a sequence of numbers 0, 1 up to the maximum input length - 1. 
- The `context_length` is a variable that represents the supported input size of the LLM. 

In [27]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, 256)
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
